# QuickBite Express - Data Quality Report
## Sprint 1, Task 1: Data Loading & Validation

**Purpose:** Load all 8 datasets, validate schema, check nulls/duplicates/dtypes, validate star schema relationships, and export a summary report.

**Output:** `/output/01_data_quality_report/`

---
## 0. Setup & Configuration

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# ============================================================
# CONFIG - Change these paths to reuse for new clients
# ============================================================
DATA_DIR = "datasets/"
OUTPUT_DIR = "output/01_data_quality_report/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CRISIS_START = "2025-06-01"
CRISIS_END = "2025-09-30"
PRE_CRISIS_START = "2025-01-01"
PRE_CRISIS_END = "2025-05-31"

print("✅ Configuration loaded")
print(f"   Data Directory: {DATA_DIR}")
print(f"   Output Directory: {OUTPUT_DIR}")
print(f"   Pre-Crisis: {PRE_CRISIS_START} to {PRE_CRISIS_END}")
print(f"   Crisis: {CRISIS_START} to {CRISIS_END}")

✅ Configuration loaded
   Data Directory: datasets/
   Output Directory: output/01_data_quality_report/
   Pre-Crisis: 2025-01-01 to 2025-05-31
   Crisis: 2025-06-01 to 2025-09-30


---
## 1. Load All Datasets

In [2]:
print("📂 Loading datasets...\n")

fact_orders = pd.read_csv(f"{DATA_DIR}fact_orders.csv")
fact_order_items = pd.read_csv(f"{DATA_DIR}fact_order_items.csv")
fact_ratings = pd.read_csv(f"{DATA_DIR}fact_ratings.csv")
fact_delivery = pd.read_csv(f"{DATA_DIR}fact_delivery_performance.csv")
dim_customer = pd.read_csv(f"{DATA_DIR}dim_customer.csv")
dim_restaurant = pd.read_csv(f"{DATA_DIR}dim_restaurant.csv")
dim_delivery_partner = pd.read_csv(f"{DATA_DIR}dim_delivery_partner_.csv")
dim_menu_item = pd.read_csv(f"{DATA_DIR}dim_menu_item.csv")

datasets = {
    "fact_orders": fact_orders,
    "fact_order_items": fact_order_items,
    "fact_ratings": fact_ratings,
    "fact_delivery_performance": fact_delivery,
    "dim_customer": dim_customer,
    "dim_restaurant": dim_restaurant,
    "dim_delivery_partner": dim_delivery_partner,
    "dim_menu_item": dim_menu_item,
}

print("✅ All 8 datasets loaded successfully!")
for name, df in datasets.items():
    print(f"   {name}: {len(df):,} rows x {df.shape[1]} cols")

📂 Loading datasets...

✅ All 8 datasets loaded successfully!
   fact_orders: 149,166 rows x 11 cols
   fact_order_items: 342,994 rows x 8 cols
   fact_ratings: 68,842 rows x 7 cols
   fact_delivery_performance: 149,166 rows x 4 cols
   dim_customer: 107,776 rows x 4 cols
   dim_restaurant: 19,995 rows x 7 cols
   dim_delivery_partner: 15,000 rows x 7 cols
   dim_menu_item: 342,671 rows x 6 cols


---
## 2. Dataset Overview

In [3]:
print("=" * 70)
print("TABLE 1: DATASET OVERVIEW")
print("=" * 70)

overview_data = []
for name, df in datasets.items():
    overview_data.append({
        "Table": name,
        "Rows": f"{len(df):,}",
        "Columns": df.shape[1],
        "Memory (MB)": round(df.memory_usage(deep=True).sum() / 1024**2, 2),
    })

overview_df = pd.DataFrame(overview_data)
print(overview_df.to_string(index=False))

TABLE 1: DATASET OVERVIEW
                    Table    Rows  Columns  Memory (MB)
              fact_orders 149,166       11        69.96
         fact_order_items 342,994        8        99.77
             fact_ratings  68,842        7        24.13
fact_delivery_performance 149,166        4        13.66
             dim_customer 107,776        4        26.88
           dim_restaurant  19,995        7         8.78
     dim_delivery_partner  15,000        7         5.53
            dim_menu_item 342,671        6       110.07


---
## 3. Null Value Analysis

In [4]:
print("=" * 70)
print("TABLE 2: NULL VALUE ANALYSIS")
print("=" * 70)

null_data = []
for name, df in datasets.items():
    null_counts = df.isnull().sum()
    null_cols = null_counts[null_counts > 0]
    if len(null_cols) > 0:
        for col, count in null_cols.items():
            null_data.append({
                "Table": name,
                "Column": col,
                "Null Count": count,
                "Null %": round(count / len(df) * 100, 2),
            })
    else:
        null_data.append({
            "Table": name,
            "Column": "None",
            "Null Count": 0,
            "Null %": 0.0,
        })

null_df = pd.DataFrame(null_data)
print(null_df.to_string(index=False))

TABLE 2: NULL VALUE ANALYSIS
                    Table              Column  Null Count  Null %
              fact_orders delivery_partner_id        5635    3.78
         fact_order_items                None           0    0.00
             fact_ratings            order_id          17    0.02
             fact_ratings         customer_id          17    0.02
             fact_ratings       restaurant_id          17    0.02
             fact_ratings              rating          17    0.02
             fact_ratings         review_text          17    0.02
             fact_ratings    review_timestamp          17    0.02
             fact_ratings     sentiment_score          17    0.02
fact_delivery_performance                None           0    0.00
             dim_customer                None           0    0.00
           dim_restaurant                None           0    0.00
     dim_delivery_partner                None           0    0.00
            dim_menu_item                None  

---
## 4. Duplicate Check

In [5]:
print("=" * 70)
print("TABLE 3: DUPLICATE CHECK")
print("=" * 70)

pk_map = {
    "fact_orders": "order_id",
    "fact_order_items": None,  # composite key (order_id + item_id)
    "fact_ratings": "order_id",
    "fact_delivery_performance": "order_id",
    "dim_customer": "customer_id",
    "dim_restaurant": "restaurant_id",
    "dim_delivery_partner": "delivery_partner_id",
    "dim_menu_item": "menu_item_id",
}

dup_data = []
for name, df in datasets.items():
    pk = pk_map[name]
    full_dups = df.duplicated().sum()
    if pk:
        pk_dups = df[pk].duplicated().sum()
    else:
        # fact_order_items: composite key
        pk_dups = df.duplicated(subset=["order_id", "item_id"]).sum()
        pk = "order_id + item_id"
    dup_data.append({
        "Table": name,
        "Primary Key": pk,
        "Full Row Duplicates": full_dups,
        "PK Duplicates": pk_dups,
        "Status": "✅ Clean" if pk_dups == 0 else "⚠️ Has Duplicates",
    })

dup_df = pd.DataFrame(dup_data)
print(dup_df.to_string(index=False))

TABLE 3: DUPLICATE CHECK
                    Table         Primary Key  Full Row Duplicates  PK Duplicates            Status
              fact_orders            order_id                    0              0           ✅ Clean
         fact_order_items  order_id + item_id                    0              0           ✅ Clean
             fact_ratings            order_id                   16             16 ⚠️ Has Duplicates
fact_delivery_performance            order_id                    0              0           ✅ Clean
             dim_customer         customer_id                    0              0           ✅ Clean
           dim_restaurant       restaurant_id                    0              0           ✅ Clean
     dim_delivery_partner delivery_partner_id                    0              0           ✅ Clean
            dim_menu_item        menu_item_id                    0              0           ✅ Clean


---
## 5. Data Type Validation

In [6]:
print("=" * 70)
print("TABLE 4: DATA TYPE CHECK")
print("=" * 70)

dtype_data = []
for name, df in datasets.items():
    for col in df.columns:
        dtype_data.append({
            "Table": name,
            "Column": col,
            "Current Dtype": str(df[col].dtype),
            "Sample Value": str(df[col].dropna().iloc[0]) if len(df[col].dropna()) > 0 else "N/A",
        })

dtype_df = pd.DataFrame(dtype_data)
print(dtype_df.to_string(index=False))

TABLE 4: DATA TYPE CHECK
                    Table                      Column Current Dtype              Sample Value
              fact_orders                    order_id        object           ORD202501023439
              fact_orders                 customer_id        object                CUST181110
              fact_orders               restaurant_id        object                 REST08622
              fact_orders         delivery_partner_id        object                   DP05541
              fact_orders             order_timestamp        object       2025-01-01 12:00:00
              fact_orders             subtotal_amount       float64                    471.62
              fact_orders             discount_amount       float64                     35.44
              fact_orders                delivery_fee       float64                     30.56
              fact_orders                total_amount       float64                    466.74
              fact_orders          

---
## 6. Star Schema Relationship Validation

In [7]:
print("=" * 70)
print("TABLE 5: STAR SCHEMA RELATIONSHIP VALIDATION")
print("=" * 70)

relationships = [
    ("fact_orders", "customer_id", "dim_customer", "customer_id"),
    ("fact_orders", "restaurant_id", "dim_restaurant", "restaurant_id"),
    ("fact_orders", "delivery_partner_id", "dim_delivery_partner", "delivery_partner_id"),
    ("fact_orders", "order_id", "fact_delivery_performance", "order_id"),
    ("fact_orders", "order_id", "fact_ratings", "order_id"),
    ("fact_order_items", "order_id", "fact_orders", "order_id"),
    ("fact_order_items", "menu_item_id", "dim_menu_item", "menu_item_id"),
]

rel_data = []
for fact_table, fact_col, dim_table, dim_col in relationships:
    fact_df = datasets[fact_table]
    dim_df = datasets[dim_table]
    
    fact_keys = set(fact_df[fact_col].dropna().unique())
    dim_keys = set(dim_df[dim_col].dropna().unique())
    
    orphan_count = len(fact_keys - dim_keys)
    match_pct = round((1 - orphan_count / len(fact_keys)) * 100, 2) if len(fact_keys) > 0 else 100
    
    rel_data.append({
        "Fact Table": fact_table,
        "FK Column": fact_col,
        "Dim Table": dim_table,
        "PK Column": dim_col,
        "Fact Unique Keys": len(fact_keys),
        "Dim Unique Keys": len(dim_keys),
        "Orphan Keys": orphan_count,
        "Match %": match_pct,
        "Status": "✅ Valid" if orphan_count == 0 else f"⚠️ {orphan_count} orphans",
    })

rel_df = pd.DataFrame(rel_data)
print(rel_df.to_string(index=False))

TABLE 5: STAR SCHEMA RELATIONSHIP VALIDATION
      Fact Table           FK Column                 Dim Table           PK Column  Fact Unique Keys  Dim Unique Keys  Orphan Keys  Match %           Status
     fact_orders         customer_id              dim_customer         customer_id            105180           107776         4930    95.31  ⚠️ 4930 orphans
     fact_orders       restaurant_id            dim_restaurant       restaurant_id             19983            19995            0   100.00          ✅ Valid
     fact_orders delivery_partner_id      dim_delivery_partner delivery_partner_id             15000            15000            0   100.00          ✅ Valid
     fact_orders            order_id fact_delivery_performance            order_id            149166           149166            0   100.00          ✅ Valid
     fact_orders            order_id              fact_ratings            order_id            149166            68825        80341    46.14 ⚠️ 80341 orphans
fact_order_it

---
## 6.1 Investigate Orphan Keys (Deep Dive)

In [8]:
print("🔍 ORPHAN KEY INVESTIGATION")
print("=" * 70)

# Check each relationship with orphans
for _, row in rel_df[rel_df["Orphan Keys"] > 0].iterrows():
    fact_table = row["Fact Table"]
    fact_col = row["FK Column"]
    dim_table = row["Dim Table"]
    dim_col = row["PK Column"]
    
    fact_df = datasets[fact_table]
    dim_df = datasets[dim_table]
    
    fact_keys = set(fact_df[fact_col].dropna().unique())
    dim_keys = set(dim_df[dim_col].dropna().unique())
    
    orphans = fact_keys - dim_keys
    
    print(f"\n📌 {fact_table}.{fact_col} -> {dim_table}.{dim_col}")
    print(f"   Orphan count: {len(orphans)}")
    print(f"   Sample orphan keys (first 5): {list(orphans)[:5]}")
    
    # How many ROWS are affected?
    affected_rows = fact_df[fact_df[fact_col].isin(orphans)].shape[0]
    print(f"   Affected rows in {fact_table}: {affected_rows:,} / {len(fact_df):,} ({round(affected_rows/len(fact_df)*100, 2)}%)")
    
    # Check if it's a 1:1 relationship issue (fact_ratings doesn't have ALL orders - that's expected)
    if dim_table == "fact_ratings":
        print(f"   ℹ️ NOTE: Not all orders have ratings. This is EXPECTED behavior.")
        print(f"   Orders with ratings: {len(dim_keys):,} / {len(fact_keys):,} ({round(len(dim_keys)/len(fact_keys)*100, 2)}%)")
    
    # Check dim_customer - some customers may not have placed orders
    if dim_table == "dim_customer":
        reverse_orphans = dim_keys - fact_keys
        print(f"   ℹ️ Customers in dim but NOT in orders: {len(reverse_orphans):,}")
        print(f"   (These are registered customers who never ordered - normal)")

🔍 ORPHAN KEY INVESTIGATION

📌 fact_orders.customer_id -> dim_customer.customer_id
   Orphan count: 4930
   Sample orphan keys (first 5): ['CUST004561', 'CUST063958', 'CUST042900', 'CUST111308', 'CUST043678']
   Affected rows in fact_orders: 5,053 / 149,166 (3.39%)
   ℹ️ Customers in dim but NOT in orders: 7,526
   (These are registered customers who never ordered - normal)

📌 fact_orders.order_id -> fact_ratings.order_id
   Orphan count: 80341
   Sample orphan keys (first 5): ['ORD202501022298', 'ORD202503015565', 'ORD202505005961', 'ORD202503013874', 'ORD202502012136']
   Affected rows in fact_orders: 80,341 / 149,166 (53.86%)
   ℹ️ NOTE: Not all orders have ratings. This is EXPECTED behavior.
   Orders with ratings: 68,825 / 149,166 (46.14%)

📌 fact_order_items.order_id -> fact_orders.order_id
   Orphan count: 16425
   Sample orphan keys (first 5): ['ORD202502005010', 'ORD202503016085', 'ORD202501009724', 'ORD202508008426', 'ORD202507002249']
   Affected rows in fact_order_items: 36,

---
## 7. Key Statistics & Distributions

In [9]:
print("=" * 70)
print("TABLE 6: KEY FIELD DISTRIBUTIONS")
print("=" * 70)

# Orders date range
fact_orders["order_timestamp"] = pd.to_datetime(fact_orders["order_timestamp"])
print(f"\n📅 Order Date Range: {fact_orders['order_timestamp'].min()} to {fact_orders['order_timestamp'].max()}")

# Monthly order counts
monthly = fact_orders.groupby(fact_orders["order_timestamp"].dt.to_period("M")).size()
print(f"\n📊 Monthly Order Counts:")
for period, count in monthly.items():
    phase = "Pre-Crisis" if str(period) <= "2025-05" else "Crisis"
    bar = "█" * int(count / 1000)
    print(f"   {period}: {count:>6,} orders [{phase:>10}] {bar}")

TABLE 6: KEY FIELD DISTRIBUTIONS

📅 Order Date Range: 2025-01-01 12:00:00 to 2025-09-30 22:59:00

📊 Monthly Order Counts:
   2025-01: 23,539 orders [Pre-Crisis] ███████████████████████
   2025-02: 22,667 orders [Pre-Crisis] ██████████████████████
   2025-03: 23,543 orders [Pre-Crisis] ███████████████████████
   2025-04: 21,466 orders [Pre-Crisis] █████████████████████
   2025-05: 22,591 orders [Pre-Crisis] ██████████████████████
   2025-06:  9,293 orders [    Crisis] █████████
   2025-07:  8,818 orders [    Crisis] ████████
   2025-08:  8,555 orders [    Crisis] ████████
   2025-09:  8,694 orders [    Crisis] ████████


In [10]:
# Cancellation check
cancel_counts = fact_orders["is_cancelled"].value_counts()
print(f"❌ Cancellation Distribution:")
for val, count in cancel_counts.items():
    print(f"   {val}: {count:,} ({round(count/len(fact_orders)*100, 2)}%)")

# COD check
cod_counts = fact_orders["is_cod"].value_counts()
print(f"\n💰 COD Distribution:")
for val, count in cod_counts.items():
    print(f"   {val}: {count:,} ({round(count/len(fact_orders)*100, 2)}%)")

❌ Cancellation Distribution:
   N: 138,054 (92.55%)
   Y: 11,112 (7.45%)

💰 COD Distribution:
   N: 102,351 (68.62%)
   Y: 46,815 (31.38%)


In [11]:
# Categorical distributions
print(f"🏙️ Cities: {sorted(dim_customer['city'].unique())}")
print(f"\n🍽️ Cuisine Types: {sorted(dim_restaurant['cuisine_type'].unique())}")
print(f"\n🏪 Partner Types: {sorted(dim_restaurant['partner_type'].unique())}")
print(f"\n🚗 Vehicle Types: {sorted(dim_delivery_partner['vehicle_type'].unique())}")
print(f"\n📢 Acquisition Channels: {sorted(dim_customer['acquisition_channel'].unique())}")

🏙️ Cities: ['Ahmedabad', 'Bengaluru', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai', 'Pune']

🍽️ Cuisine Types: ['Biryani', 'Chinese', 'Desserts', 'Fast Food', 'Healthy', 'North Indian', 'Pizza', 'South Indian']

🏪 Partner Types: ['Cloud Kitchen', 'Restaurant']

🚗 Vehicle Types: ['Bike', 'Car', 'Cycle', 'Scooter']

📢 Acquisition Channels: ['Organic', 'Paid', 'Referral', 'Social']


In [12]:
# Ratings & sentiment
print(f"⭐ Rating Range: {fact_ratings['rating'].min()} to {fact_ratings['rating'].max()}")
print(f"📝 Sentiment Score Range: {fact_ratings['sentiment_score'].min()} to {fact_ratings['sentiment_score'].max()}")
print(f"📝 Reviews Available: {fact_ratings['review_text'].notna().sum():,} / {len(fact_ratings):,}")

⭐ Rating Range: 1.0 to 5.0
📝 Sentiment Score Range: -1.0 to 1.0
📝 Reviews Available: 68,825 / 68,842


---
## 8. Numeric Field Statistics

In [13]:
print("=" * 70)
print("TABLE 7: NUMERIC FIELD STATISTICS (fact_orders)")
print("=" * 70)

numeric_cols = ["subtotal_amount", "discount_amount", "delivery_fee", "total_amount"]
stats = fact_orders[numeric_cols].describe().round(2)
print(stats.to_string())

TABLE 7: NUMERIC FIELD STATISTICS (fact_orders)
       subtotal_amount  discount_amount  delivery_fee  total_amount
count        149166.00        149166.00     149166.00     149166.00
mean            314.37            18.89         30.07        325.55
std             147.35            29.80         11.00        146.79
min               0.00             0.00          0.00          0.00
25%             234.96             0.00         24.73        249.84
50%             309.07             0.00         31.48        321.70
75%             396.29            34.69         38.24        406.59
max             900.00           222.53         45.00        944.91


In [14]:
print("=" * 70)
print("TABLE 8: DELIVERY PERFORMANCE STATISTICS")
print("=" * 70)

del_stats = fact_delivery[["actual_delivery_time_mins", "expected_delivery_time_mins", "distance_km"]].describe().round(2)
print(del_stats.to_string())

TABLE 8: DELIVERY PERFORMANCE STATISTICS
       actual_delivery_time_mins  expected_delivery_time_mins  distance_km
count                  149166.00                    149166.00    149166.00
mean                       44.40                        38.69         4.49
std                        12.78                         5.07         2.02
min                        25.00                        30.00         1.00
25%                        36.00                        35.00         2.70
50%                        42.00                        39.00         4.50
75%                        50.00                        43.00         6.20
max                        90.00                        50.00         8.00


---
## 9. Data Quality Score

In [15]:
print("=" * 70)
print("📋 DATA QUALITY SUMMARY")
print("=" * 70)

total_nulls = sum(df.isnull().sum().sum() for df in datasets.values())
total_cells = sum(df.shape[0] * df.shape[1] for df in datasets.values())
completeness = round((1 - total_nulls / total_cells) * 100, 2)

total_pk_dups = sum(row["PK Duplicates"] for _, row in dup_df.iterrows())
total_orphans = sum(row["Orphan Keys"] for _, row in rel_df.iterrows())

# Adjusted orphan count: exclude fact_orders -> fact_ratings (expected: not all orders have ratings)
expected_orphan_row = rel_df[(rel_df["Fact Table"] == "fact_orders") & (rel_df["Dim Table"] == "fact_ratings")]
expected_orphans = expected_orphan_row["Orphan Keys"].values[0] if len(expected_orphan_row) > 0 else 0
adjusted_orphans = total_orphans - expected_orphans

print(f"\n   Data Completeness:         {completeness}%")
print(f"   Total Null Values:         {total_nulls:,}")
print(f"   PK Duplicates Found:       {total_pk_dups:,}")
print(f"   Total Orphan Keys:         {total_orphans:,}")
print(f"   Expected Orphans (ratings): {expected_orphans:,} (not all orders get rated)")
print(f"   Unexpected Orphans:        {adjusted_orphans:,}")
print(f"   Tables Validated:          {len(datasets)}/8")
print(f"   Relationships Checked:     {len(relationships)}/7")

# Adjusted quality score
quality_score = completeness
if total_pk_dups > 0:
    quality_score -= min(5, total_pk_dups * 0.1)  # small penalty for few dups
if adjusted_orphans > 0:
    quality_score -= min(5, adjusted_orphans / 1000)  # capped penalty

print(f"\n   🏆 OVERALL DATA QUALITY SCORE: {round(quality_score, 1)}%")
if quality_score >= 95:
    print("   ✅ GO/NO-GO: PROCEED TO ANALYSIS")
elif quality_score >= 85:
    print("   ⚠️ GO/NO-GO: PROCEED WITH CAUTION - Document known issues")
else:
    print("   ❌ GO/NO-GO: REVIEW ISSUES BEFORE PROCEEDING")

📋 DATA QUALITY SUMMARY

   Data Completeness:         99.93%
   Total Null Values:         5,754
   PK Duplicates Found:       16
   Total Orphan Keys:         101,696
   Expected Orphans (ratings): 80,341 (not all orders get rated)
   Unexpected Orphans:        21,355
   Tables Validated:          8/8
   Relationships Checked:     7/7

   🏆 OVERALL DATA QUALITY SCORE: 93.3%
   ⚠️ GO/NO-GO: PROCEED WITH CAUTION - Document known issues


---
## 10. Export Reports

In [16]:
# Export to CSV
overview_df.to_csv(f"{OUTPUT_DIR}01_dataset_overview.csv", index=False)
null_df.to_csv(f"{OUTPUT_DIR}02_null_analysis.csv", index=False)
dup_df.to_csv(f"{OUTPUT_DIR}03_duplicate_check.csv", index=False)
rel_df.to_csv(f"{OUTPUT_DIR}04_relationship_validation.csv", index=False)
dtype_df.to_csv(f"{OUTPUT_DIR}05_dtype_check.csv", index=False)

# Export full summary as markdown (no tabulate dependency)
with open(f"{OUTPUT_DIR}DATA_QUALITY_REPORT.md", "w", encoding="utf-8") as f:
    f.write("# QuickBite Express - Data Quality Report\n\n")
    f.write(f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write(f"**Data Quality Score:** {round(quality_score, 1)}%\n\n")
    f.write("---\n\n")
    
    f.write("## Dataset Overview\n\n")
    f.write(overview_df.to_csv(index=False))
    
    f.write("\n## Null Analysis\n\n")
    f.write(null_df.to_csv(index=False))
    
    f.write("\n## Duplicate Check\n\n")
    f.write(dup_df.to_csv(index=False))
    
    f.write("\n## Relationship Validation\n\n")
    f.write(rel_df.to_csv(index=False))
    
    f.write("\n---\n")
    f.write(f"\n**Completeness:** {completeness}%\n")
    f.write(f"**PK Duplicates:** {total_pk_dups}\n")
    f.write(f"**Unexpected Orphans:** {adjusted_orphans}\n")
    f.write(f"\n**Status:** {'PROCEED TO ANALYSIS' if quality_score >= 85 else 'REVIEW ISSUES'}\n")

print(f"📁 Reports exported to: {OUTPUT_DIR}")
print("   - 01_dataset_overview.csv")
print("   - 02_null_analysis.csv")
print("   - 03_duplicate_check.csv")
print("   - 04_relationship_validation.csv")
print("   - 05_dtype_check.csv")
print("   - DATA_QUALITY_REPORT.md")
print("\n✅ Sprint 1, Task 1 COMPLETE!")

📁 Reports exported to: output/01_data_quality_report/
   - 01_dataset_overview.csv
   - 02_null_analysis.csv
   - 03_duplicate_check.csv
   - 04_relationship_validation.csv
   - 05_dtype_check.csv
   - DATA_QUALITY_REPORT.md

✅ Sprint 1, Task 1 COMPLETE!


---
## Summary & Next Steps

**Key Findings:**
- Not all orders have ratings — this is expected (fact_ratings is a subset)
- Orphan keys from orders->ratings are NOT data quality issues
- Check orphan investigation section above for any real issues

**Next:** Sprint 1, Task 2 → Star Schema Implementation + EDA (Q1, Q2, Q3, Q8)